In [60]:
import os;
import json;
import builtins
import pandas as pd;

dir = '/home/chesetti/Repos/learned_merge_cleanup/sponge'
testcase = 'osm_merge_small_1'
test_dir = os.path.join(dir, testcase)
results_dir = os.path.join(test_dir, 'outputs', 'results')
csv_dir = os.path.join(test_dir, 'csv')
os.makedirs(csv_dir, exist_ok=True)

runs = [os.path.join(results_dir, run) for run in os.listdir(results_dir)]
print(runs)

['/home/chesetti/Repos/learned_merge_cleanup/sponge/wiki_merge_small_1/outputs/results/run0', '/home/chesetti/Repos/learned_merge_cleanup/sponge/wiki_merge_small_1/outputs/results/run1', '/home/chesetti/Repos/learned_merge_cleanup/sponge/wiki_merge_small_1/outputs/results/run2']


In [61]:
# Walk all the json files and put them in a dataframe

test_results = []
for run in runs:
    for test_result_file in os.listdir(run):
        json_file = builtins.open(os.path.join(run, test_result_file))
        test_result = json.load(json_file)
        test_result['run'] = run
        test_results.append(test_result)
        json_file.close()
test_dataframe = pd.json_normalize(test_results)
print(test_dataframe.columns)

        

Index(['command', 'run', 'result.checksum', 'result.duration_ns',
       'result.duration_sec', 'result.inner_disk_fetch',
       'result.inner_index_build_duration_ns', 'result.inner_index_size',
       'result.outer_disk_fetch', 'spec.algo', 'spec.algo_name',
       'spec.common_key', 'spec.index.search', 'spec.index.type',
       'spec.inner_table', 'spec.key_size', 'spec.key_type',
       'spec.load_sstable_in_mem', 'spec.name', 'spec.num_threads',
       'spec.outer_table', 'spec.result_path', 'spec.value_size',
       'spec.write_result_to_disk', 'spec.index.leaf_size_in_pages'],
      dtype='object')


# Reports

### Overall Duration

In [62]:
overall_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.duration_ns', aggfunc='median')
overall_duration.to_csv(os.path.join(csv_dir, 'duration_sec.csv'))
print(overall_duration)

spec.algo_name      BTree   PGM1024    PGM256     PGM64  standard_merge
spec.common_key                                                        
10               45945151  34020956  35897755  36165195        53054879
20               33331832  26344045  26960142  28134965        49801661
30               30104509  23497990  24178520  24455250        47468195
40               27841982  22238694  22616706  23011005        47126154
50               25929593  21450914  21786595  22142141        46396541
60               24611161  20903339  21012812  21447713        46291065
70               25550913  20440435  21203598  20990173        46354259
80               23940477  20038197  20379034  20622475        50772803
90               23263890  19919146  20020745  20234447        47299097
100              22894481  19645943  19752015  20080803        45667093


In [63]:
inner_index_size = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_size', aggfunc='median')
data = {}
data['indexes'] = []
data['memory_bytes'] = []
for index in inner_index_size:
    data['indexes'].append(index)
    data['memory_bytes'].append(inner_index_size[index].mean())
inner_index_size = pd.DataFrame(data=data)
inner_index_size.to_csv(os.path.join(csv_dir, 'inner_index_size.csv'))
print(inner_index_size)

          indexes  memory_bytes
0           BTree       12352.0
1         PGM1024         328.0
2          PGM256         896.0
3           PGM64        9264.0
4  standard_merge       41088.0


In [64]:
inner_index_build_duration = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_index_build_duration_ns', aggfunc='median')
data = {}
data['indexes'] = []
data['build_duration'] = []
for index in inner_index_build_duration:
    data['indexes'].append(index)
    data['build_duration'].append(inner_index_build_duration[index].mean())
inner_index_build_duration = pd.DataFrame(data=data)
inner_index_build_duration.to_csv(os.path.join(csv_dir, 'inner_index_build_duration_ns.csv'))

In [65]:
inner_index_fetch = test_dataframe.pivot_table(index='spec.common_key', columns='spec.algo_name', values='result.inner_disk_fetch', aggfunc='median')
print(inner_index_fetch)

spec.algo_name   BTree  PGM1024  PGM256  PGM64  standard_merge
spec.common_key                                               
10                3897     3897    3897   3897            3897
20                3897     3897    3897   3897            3897
30                3897     3897    3897   3897            3897
40                3897     3897    3897   3897            3897
50                3897     3897    3897   3897            3897
60                3897     3897    3897   3897            3897
70                3897     3897    3897   3897            3897
80                3897     3897    3897   3897            3897
90                3897     3897    3897   3897            3897
100               3897     3897    3897   3897            3897


In [66]:
result_checksum = (test_dataframe[['spec.common_key', 'spec.algo_name', 'result.checksum']].sort_values(by=['spec.common_key', 'spec.algo_name'])) #.loc[test_dataframe['spec.common_key']=='10'])
for common_key in sorted(test_dataframe['spec.common_key'].unique()):
    checksums = result_checksum.loc[result_checksum['spec.common_key'] == common_key]
    unique_checksums = checksums['result.checksum'].unique()
    if (len(unique_checksums) == 1):
        print(f"common_key: {common_key} checksum: {unique_checksums} OK")
    else:
        print(f"common_key: {common_key} checksums don't match")
print(result_checksum.loc[result_checksum['spec.common_key'] == 40])
print(result_checksum.loc[result_checksum['spec.common_key'] == 40]['result.checksum'].unique())

common_key: 10 checksum: ['1CCCD0E7498C9710267C368E6A9A441A'] OK
common_key: 20 checksum: ['765B4B4DA7B211F362A666308730F1B0'] OK
common_key: 30 checksum: ['4819FA57EEFB140B248A5338DA1C5741'] OK
common_key: 40 checksum: ['07F4237B4B8A197763DCA9FBF19C1883'] OK
common_key: 50 checksum: ['0AEC67B4CAA5C5556084EF0264C99955'] OK
common_key: 60 checksum: ['0CFD8469BD9D4A58F585700E3ACD5021'] OK
common_key: 70 checksum: ['D7762C6E8E16571C5FE1453D11C21002'] OK
common_key: 80 checksum: ['CB67B0C284512E25C3AEF9579E4DE03A'] OK
common_key: 90 checksum: ['6C6844E380AD9932C9F20BAEE9CDE56B'] OK
common_key: 100 checksum: ['72DB0875CE5603FA096483EB9CC02FF7'] OK
     spec.common_key  spec.algo_name                   result.checksum
6                 40           BTree  07F4237B4B8A197763DCA9FBF19C1883
56                40           BTree  07F4237B4B8A197763DCA9FBF19C1883
106               40           BTree  07F4237B4B8A197763DCA9FBF19C1883
40                40         PGM1024  07F4237B4B8A197763DCA9FBF19